### Import Necessary Libraries

In [24]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)

from peft import (
    LoraConfig,
    PeftModel,
    PeftConfig,
    prepare_model_for_kbit_training,
    get_peft_model
)

from accelerate import infer_auto_device_map, init_empty_weights

import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format, SFTConfig
from dotenv import load_dotenv
import bitsandbytes as bnb
from pathlib import Path

### Huggingface and WandB authentication

In [2]:
load_dotenv()

HF_KEY = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [3]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_KEY

In [4]:
os.environ["WANDB_NOTEBOOK_NAME"] = "C:/Users/User/Data Science/Deep Learning/Generative AI/Fine Tuning LLMs/fine-tuning llama 3.2 1B/research.ipynb"

In [5]:
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


True

In [6]:
run = wandb.init(
    project='Fine-tune Llama 3.2 on Customer Support Dataset',
    job_type="training",
    anonymous="allow"
)

In [5]:
base_model = "meta-llama/Llama-3.2-1B-Instruct"
new_model = "llama-3.2-3b-it-Ecommerce-ChatBot"
dataset_name = "bitext/Bitext-customer-support-llm-chatbot-training-dataset"

### Set the data type and attention implementation

In [7]:
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    torch_dtype = torch.bfloat16
    attn_implementation = "flash_attention_2"
else:
    torch_dtype = torch.float16
    attn_implementation = "eager"

### QLoRA Config

In [8]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True
)

### Load Model

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\accelerate\utils\modeling.py:856: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  _ = torch.tensor([0], device=i)


### Load Tokenizer

In [10]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

### Importing the dataset

In [11]:
dataset = load_dataset(dataset_name, split="train")

FIrst, we will train with only 1000 samples of the dataset

In [ ]:
train_dataset = dataset.shuffle(seed=65).select(range(1000))

In [87]:
test_dataset = dataset.shuffle(seed=65).select(range(1000, 1200))

In [88]:
len(test_dataset)

200

In [36]:
len(dataset['instruction'])

1000

### Data Preprocessing to match our chat template

In [12]:
instruction = """You are a top-rated customer service agent named John. 
    Be polite to customers and answer all their questions.
    If the question is out of context and is not related to an e-commerce business, let them know you can not help and that they should look elsewhere for answers.
    """

In [13]:
def format_chat_template(row):
    from transformers import AutoTokenizer

    base_model = "meta-llama/Llama-3.2-1B-Instruct"
    tokenizer = AutoTokenizer.from_pretrained(
        base_model, trust_remote_code=True)

    instruction = """You are a top-rated customer service agent named John. 
        Be polite to customers and answer all their questions.
        If the question is out of context and is not related to an e-commerce business, let them know you can not help and that they should look elsewhere for answers.
        """

    row_json = [{"role": "system", "content": instruction},
                {"role": "user", "content": row["instruction"]},
                {"role": "assistant", "content": row["response"]}]

    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

In [52]:
dataset = dataset.map(format_chat_template, num_proc=2)

Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [89]:
test_dataset = test_dataset.map(format_chat_template, num_proc=2)

Map (num_proc=2):   0%|          | 0/200 [00:00<?, ? examples/s]

In [69]:
dataset.push_to_hub("customer-support-1k", token=HF_KEY)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Victorano/customer-support-1k/commit/157c1da2fb0e842461161ea69d8551fc26715b4d', commit_message='Upload dataset', commit_description='', oid='157c1da2fb0e842461161ea69d8551fc26715b4d', pr_url=None, pr_revision=None, pr_num=None)

In [90]:
test_dataset.push_to_hub("customer-support-test-200", token=HF_KEY)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Victorano/customer-support-test-200/commit/af6cd0525f81ae93586b3035fb3e5adb819a651b', commit_message='Upload dataset', commit_description='', oid='af6cd0525f81ae93586b3035fb3e5adb819a651b', pr_url=None, pr_revision=None, pr_num=None)

In [33]:
dataset['text'][3]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nYou are a top-rated customer service agent named John. \n        Be polite to customers and answer all their questions.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nwhere do i enter a different shipping address<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAh, I understand that you\'re looking to enter a different shipping address. Allow me to guide you through the process:\n\n1. Log in to your account on our website.\n2. Navigate to the "My Account" or "Profile" section, which can usually be found in the top right corner of the page.\n3. Look for the "Shipping Addresses" or similar option. It may also be labeled as "Manage Addresses" or "Delivery Information."\n4. Click on that option to access your saved addresses.\n5. To enter a different shipping address, you\'ll most likely have the choice to either "Edit" an existing address or "Add

### Downloading the saved datasets from HF

In [12]:
dataset = load_dataset("Victorano/customer-support-1k", split="train")

In [29]:
dataset['text'][999]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nYou are a top-rated customer service agent named John. \n        Be polite to customers and answer all their questions.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nchange to {{Account Type}} acount<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThank you for expressing interest in upgrading to the {{Account Type}} account! I\'d be delighted to guide you through the process:\n\n1. Access Your Account: Start by logging in to our platform using your credentials.\n2. Go to Account Settings: Once you\'re logged in, navigate to the "Account Settings" section.\n3. Upgrade Option: Look for the option that allows you to upgrade your account type.\n4. Select {{Account Type}} Account: Choose the "{{Account Type}} Account" option from the available choices.\n5. Follow the Prompts: Follow the prompts on the screen to complete the upgrade process succe

In [15]:
test_dataset = load_dataset("Victorano/customer-support-test-200", split="train")

In [28]:
test_dataset['text'][0]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nYou are a top-rated customer service agent named John. \n        Be polite to customers and answer all their questions.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nis it possible to buy several items<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThank you for your interest in purchasing several items from us! We have a wide range of products available for you to choose from. To place your order, simply browse through our online catalog and add the items you want to your shopping cart. Once you've selected all the items you wish to purchase, proceed to the checkout page and follow the prompts to complete your order. If you have any questions or need assistance during the process, feel free to reach out to our customer service team who will be more than happy to help. Happy shopping!<|eot_id|>"

In [30]:
len(dataset), len(test_dataset)

(1000, 200)

### Setting Up the Model
extract the linear model name from the model

In [31]:
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [32]:
modules = find_all_linear_names(model)

In [33]:
modules

['down_proj', 'o_proj', 'up_proj', 'q_proj', 'v_proj', 'gate_proj', 'k_proj']

Using the Linear Module name to create the LoRA Adapter, this is only what we will be fine tuning

### Lora Config

In [34]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules
)

In [35]:
model, tokenizer = setup_chat_format(model, tokenizer)
model = get_peft_model(model, peft_config)

### SFT Hyperparameter

In [36]:
training_arguments = SFTConfig(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    eval_strategy="steps",
    max_seq_length=512,
    dataset_text_field="text",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)

### Supervised Fine Tuning

In [37]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    eval_dataset=test_dataset,
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False
)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


  0%|          | 0/500 [00:00<?, ?it/s]

{'loss': 2.0753, 'grad_norm': 2.98009991645813, 'learning_rate': 2e-05, 'epoch': 0.0}
{'loss': 2.196, 'grad_norm': 3.350388288497925, 'learning_rate': 4e-05, 'epoch': 0.0}
{'loss': 2.4107, 'grad_norm': 3.648029088973999, 'learning_rate': 6e-05, 'epoch': 0.01}
{'loss': 2.463, 'grad_norm': 3.2274019718170166, 'learning_rate': 8e-05, 'epoch': 0.01}
{'loss': 2.3088, 'grad_norm': 2.405561923980713, 'learning_rate': 0.0001, 'epoch': 0.01}
{'loss': 2.1519, 'grad_norm': 2.094228506088257, 'learning_rate': 0.00012, 'epoch': 0.01}
{'loss': 1.892, 'grad_norm': 1.8950233459472656, 'learning_rate': 0.00014, 'epoch': 0.01}
{'loss': 1.9028, 'grad_norm': 1.8293263912200928, 'learning_rate': 0.00016, 'epoch': 0.02}
{'loss': 1.7341, 'grad_norm': 1.8322831392288208, 'learning_rate': 0.00018, 'epoch': 0.02}
{'loss': 1.7392, 'grad_norm': 1.7804890871047974, 'learning_rate': 0.0002, 'epoch': 0.02}
{'loss': 1.3917, 'grad_norm': 1.875309944152832, 'learning_rate': 0.0001995918367346939, 'epoch': 0.02}
{'loss'

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


{'loss': 0.7397, 'grad_norm': 1.674275517463684, 'learning_rate': 0.00016326530612244898, 'epoch': 0.2}


...

  0%|          | 0/200 [00:00<?, ?it/s]

{'eval_loss': 0.6420578956604004, 'eval_runtime': 313.0002, 'eval_samples_per_second': 0.639, 'eval_steps_per_second': 0.639, 'epoch': 0.8}
{'loss': 0.8117, 'grad_norm': 0.9699596166610718, 'learning_rate': 4.0408163265306124e-05, 'epoch': 0.8}
{'loss': 0.8507, 'grad_norm': 0.972296416759491, 'learning_rate': 4e-05, 'epoch': 0.8}
{'loss': 0.7629, 'grad_norm': 1.0452474355697632, 'learning_rate': 3.9591836734693876e-05, 'epoch': 0.81}
{'loss': 0.6485, 'grad_norm': 1.1325629949569702, 'learning_rate': 3.9183673469387755e-05, 'epoch': 0.81}
{'loss': 0.8584, 'grad_norm': 1.2231523990631104, 'learning_rate': 3.8775510204081634e-05, 'epoch': 0.81}
{'loss': 0.6868, 'grad_norm': 1.1652140617370605, 'learning_rate': 3.836734693877551e-05, 'epoch': 0.81}
{'loss': 0.6327, 'grad_norm': 1.1628273725509644, 'learning_rate': 3.795918367346939e-05, 'epoch': 0.81}
{'loss': 0.6203, 'grad_norm': 1.1739225387573242, 'learning_rate': 3.7551020408163264e-05, 'epoch': 0.82}
{'loss': 0.7557, 'grad_norm': 1.28

  0%|          | 0/200 [00:00<?, ?it/s]

{'eval_loss': 0.6263864636421204, 'eval_runtime': 313.2449, 'eval_samples_per_second': 0.638, 'eval_steps_per_second': 0.638, 'epoch': 1.0}


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\peft\utils\save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


### Testing the Fine Tuned Model

#### Getting some test data

In [49]:
eval_dataset = dataset.shuffle(seed=65).select(range(1200, 1205))

In [16]:
eval_dataset[0]

{'flags': 'BELMQZ',
 'instruction': 'i do not know how to inform of a trouble withpayments',
 'category': 'PAYMENT',
 'intent': 'payment_issue',
 'response': "Don't worry, I'm here to guide you through the process of reporting any trouble you're experiencing with your payments. To inform us about the issue, you can reach out to our customer support team via phone or chat. They are available {{Customer Support Hours}} at {{Customer Support Phone Number}} or through the Live Chat on our website at {{Website URL}}. Our dedicated team will assist you step-by-step in notifying us about the trouble, ensuring that it gets resolved as quickly as possible. We appreciate you bringing this to our attention, and we want to assure you that we're committed to providing you with the best assistance for your payment concerns."}

#### Getting The Lora Model

In [25]:
peft_model_id = "C:/Users/User/Data Science/Deep Learning/Generative AI/Fine Tuning LLMs/fine-tuning llama 3.2 1B/llama-3.2-3b-it-Ecommerce-ChatBot/checkpoint-500"

In [22]:
config = PeftConfig.from_pretrained(peft_model_id)

In [28]:
lora_model = PeftModel(model=model, peft_config=config)

In [30]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2048, out_features=512, bias=False)
            (lora_dropout): ModuleDict(
       

In [29]:
lora_model

PeftModel(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bi

#### Preprocessing the eval data

In [61]:
instruction = """You are a top-rated customer service agent named John. 
    Be polite to customers and answer all their questions.
    If the question is out of context and is not related to an e-commerce business, let them know you can not help and that they should look elsewhere for answers.
    """

messages = [{"role": "system", "content": instruction},
            {"role": "user", "content": "I have to see what payment payment modalities are accepted"}]

In [33]:
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [46]:
prompt

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nYou are a top-rated customer service agent named John. \n    Be polite to customers and answer all their questions.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nI have to see what payment payment modalities are accepted<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'

In [47]:
def format_eval_template(row):
    from transformers import AutoTokenizer

    base_model = "meta-llama/Llama-3.2-1B-Instruct"
    tokenizer = AutoTokenizer.from_pretrained(
        base_model, trust_remote_code=True)

    instruction = """You are a top-rated customer service agent named John. 
        Be polite to customers and answer all their questions.
        If the question is out of context and is not related to an e-commerce business, let them know you can not help and that they should look elsewhere for answers.
        """

    row_json = [{"role": "system", "content": instruction},
                {"role": "user", "content": row["instruction"]}]

    prompt = tokenizer.apply_chat_template(row_json, tokenize=False, add_generation_prompt=True)
    return prompt

In [51]:
eval_prompt_1 = format_eval_template(eval_dataset[0])

In [53]:
eval_dataset['instruction'][0]

'i do not know how to inform of a trouble withpayments'

In [52]:
eval_prompt_1

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nYou are a top-rated customer service agent named John. \n        Be polite to customers and answer all their questions.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\ni do not know how to inform of a trouble withpayments<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'

In [62]:
def preprocess_eval_data_and_pred(prompt):
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True).to("cuda")
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    outputs = lora_model.generate(**inputs, max_new_tokens=150, num_return_sequences=1)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text

In [45]:
eval_dataset[0]

{'flags': 'BELMQZ',
 'instruction': 'i do not know how to inform of a trouble withpayments',
 'category': 'PAYMENT',
 'intent': 'payment_issue',
 'response': "Don't worry, I'm here to guide you through the process of reporting any trouble you're experiencing with your payments. To inform us about the issue, you can reach out to our customer support team via phone or chat. They are available {{Customer Support Hours}} at {{Customer Support Phone Number}} or through the Live Chat on our website at {{Website URL}}. Our dedicated team will assist you step-by-step in notifying us about the trouble, ensuring that it gets resolved as quickly as possible. We appreciate you bringing this to our attention, and we want to assure you that we're committed to providing you with the best assistance for your payment concerns."}

In [54]:
text = preprocess_eval_data_and_pred(eval_prompt_1)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [55]:
print(text.split("assistant")[1])



I'm here to help. Don't worry, it's completely normal to have questions or concerns about your payments. Can you please tell me more about what's not working as expected with your payments? What kind of issue are you experiencing? Is it a payment that's been delayed, missed, or something else?


In [56]:
message_1 = [{"role": "system", "content": instruction},
            {"role": "user", "content": "what is your name and what do you do"}]

In [57]:
prompt_1 = tokenizer.apply_chat_template(message_1, tokenize=False, add_generation_prompt=True)

In [58]:
eval_prompt_2 = preprocess_eval_data_and_pred(prompt_1)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [60]:
print(eval_prompt_2.split("assistant")[1])



My name is John, and I'm a dedicated customer service representative. I'm here to help you with any questions or concerns you may have. How can I assist you today?


In [63]:
message_2 = [{"role": "system", "content": instruction},
             {"role": "user", "content": "what do you know about wind and air"}]

In [64]:
prompt_2 = tokenizer.apply_chat_template(
    message_2, tokenize=False, add_generation_prompt=True)

In [65]:
response = preprocess_eval_data_and_pred(prompt_2)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [68]:
print(response.split("assistant")[1])



I'd be happy to help you with any questions you have about wind and air.

Wind and air are fascinating topics that have been studied extensively in various fields, including physics, meteorology, and engineering.

**What is wind?**

Wind is the movement of air in the atmosphere, which is created by the uneven heating of the Earth's surface by the sun. The air rises in areas of high temperature and sinks in areas of low temperature. This movement of air is known as convection.

**Types of wind:**

There are several types of wind, including:

1. **Trade winds**: These are winds that blow from the equator towards the poles.
2. **Westerlies**: These are winds that blow from the west towards the
